In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from astropy.table import Table, vstack, Column
import os
import pdb
import warnings
import sys
import pandas as pd
from pandas import DataFrame as df

working_dir = '/Users/Gianna/Documents/GitHub/SIP2019/'

sys.path

['',
 '/Users/Gianna/Documents/GitHub/SIP2019/Giovanni',
 '/Users/Gianna/anaconda3/lib/python37.zip',
 '/Users/Gianna/anaconda3/lib/python3.7',
 '/Users/Gianna/anaconda3/lib/python3.7/lib-dynload',
 '/Users/Gianna/anaconda3/lib/python3.7/site-packages',
 '/Users/Gianna/anaconda3/lib/python3.7/site-packages/aeosa',
 '/Users/Gianna/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/Users/Gianna/.ipython']

In [2]:
objtypes=[]
objfiles=[]
path1 = './../spectra/'

bigFile = Table.read('bigfile-Copy1.fits')
allnames=list(bigFile['SPEC1DNAME'])

for filename in os.listdir(path1):
    if ((filename in allnames) | (filename [:-3] in allnames)):
        if ('[]' not in str((bigFile[bigFile['SPEC1DNAME']==filename])['Type'])):
            objtypes.append(str((bigFile[bigFile['SPEC1DNAME']==filename])['Type']))
        else:
            objtypes.append(str((bigFile[bigFile['SPEC1DNAME']==filename[:-3]])['Type']))
        objfiles.append(filename)

degc, star, icl, udg, halo, denu = [],[],[],[],[],[]
for i in range(len(objfiles)):
    if('DEGC'in objtypes[i]):
        degc.append(objfiles[i])
    elif('ICL' in objtypes[i]):
        icl.append(objfiles[i])
    elif('UDG'in objtypes[i]):
        udg.append(objfiles[i] +'.gz')
    elif('Halo' in objtypes[i]):
        halo.append(objfiles[i])
    elif('DENU' in objtypes[i]):
        denu.append(objfiles[i])

In [3]:
sys.path.append(working_dir + 'lib')

from py_specrebin import rebinspec

correct = np.arange(4000,8750.9,0.9)

i = 0

In [5]:
vobsDEGC, vobsHalo, vobsDENU, vobsUDG, vobsICL = [],[],[],[],[]

def vobs(objfiles):
    vobszero = np.zeros(len(objfiles))
    for i in range(len(objfiles)):
        vobszero[i]=(float(bigFile[bigFile['SPEC1DNAME'] == objfiles[i]]['VOBS']))
    return vobszero

vobsDEGC = vobs(degc)
vobsHalo = vobs(halo)
vobsDENU = vobs(denu)
vobsUDG = vobs(udg)
vobsICL = vobs(icl)

ValueError: can only convert an array of size 1 to a Python scalar